In [1]:
! pip install emoji

     |████████████████████████████████| 51kB 2.0MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=dbf4c588c0062cff681a2151ee3f2465aa4dbc9952168570b473bba0debd3cc8
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [2]:
import emoji
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":grinning_face_with_big_eyes:",
                    "3": ":zipper-mouth_face:",
                    "4": ":fork_and_knife:",
                    "5": ":tongue:",
                    "6": ":sunglasses:",
                    "7": ":kiss:",
                    "8": ":winking_face_with_tongue:",
                   }

In [4]:
for key in emoji_dictionary.keys():
    print(f"{key} ----> {emoji_dictionary[key]}")
    print(emoji.emojize(emoji_dictionary[key]))

0 ----> ❤️
❤️
1 ----> :baseball:
⚾
2 ----> :grinning_face_with_big_eyes:
😃
3 ----> :zipper-mouth_face:
🤐
4 ----> :fork_and_knife:
🍴
5 ----> :tongue:
👅
6 ----> :sunglasses:
🕶
7 ----> :kiss:
💏
8 ----> :winking_face_with_tongue:
😜


In [5]:
# lib
import numpy as np
import pandas as pd

In [8]:
train = pd.read_csv("train_emoji.csv", header= None)
test = pd.read_csv("test_emoji.csv", header= None)

In [9]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [10]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [11]:
train.drop([2, 3], inplace= True, axis = 1)

In [12]:
train.head()

,0,1
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       132 non-null    object
 1   1       132 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


In [14]:
train.shape

(132, 2)

In [33]:
X_train = train.iloc[:, 0].values
y_train = train.iloc[:, 1].values

In [34]:
X_train.shape, y_train.shape

((132,), (132,))

In [35]:
X_train[:5]

array(['never talk to me again', 'I am proud of your achievements',
       'It is the worst day in my life', 'Miss you so much',
       'food is life'], dtype=object)

In [36]:
y_train[:5]

array([3, 2, 3, 0, 4])

In [21]:
for i in range(5):
    print(X_train[i], end = " ---> ")
    print(emoji.emojize(emoji_dictionary[str(y_train[i])]))

never talk to me again ---> 🤐
I am proud of your achievements ---> 😃
It is the worst day in my life ---> 🤐
Miss you so much ---> ❤️
food is life ---> 🍴


In [22]:
np.unique(y_train)

array([0, 1, 2, 3, 4])

In [37]:
y_train = y_train.reshape(-1, 1) # (132, 1)
y_train.shape

(132, 1)

In [38]:
# onehot encoding
from sklearn.preprocessing import OneHotEncoder
one_y = OneHotEncoder()

y_train = one_y.fit_transform(y_train).toarray()

In [39]:
y_train.shape

(132, 5)

In [40]:
y_train[:5]

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [41]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"smagni","key":"806ed6aae3534efc2f7b44fcc46b4a12"}'}

In [42]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [43]:
!kaggle datasets download -d watts2/glove6b50dtxt

 90% 61.0M/67.7M [00:02<00:00, 19.8MB/s]
100% 67.7M/67.7M [00:02<00:00, 28.7MB/s]


In [44]:
!unzip '/content/glove6b50dtxt.zip'

Archive:  /content/glove6b50dtxt.zip
  inflating: glove.6B.50d.txt        


In [50]:
emb = {}

with open('/content/glove.6B.50d.txt', 'r') as f:
    for line in f:
        # print(line.split())
        line = line.split()
        word = line[0]
        value = line[1:]
        value = np.asarray(value, dtype= 'float32')
        emb[word] = value

In [52]:
def to_get_outputEmb(X):
    emb_mat_out = np.zeros((X.shape[0], 10, 50))

    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for ij in range(len(X[ix])):
            emb_mat_out[ix][ij] = emb[X[ix][ij].lower()]
        
    return emb_mat_out

In [53]:
X_train = to_get_outputEmb(X_train)

In [54]:
X_train.shape

(132, 10, 50)

## NN

In [55]:
from keras import Sequential
from keras.layers import *

In [61]:
model = Sequential()

model.add(LSTM(64, input_shape = (10, 50), return_sequences= True))
model.add(Dropout(rate= 0.3))
model.add(LSTM(64, return_sequences= False))
model.add(Dropout(rate= 0.3))
model.add(Dense(5, activation= 'softmax'))

In [62]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [63]:
model.compile(optimizer= 'rmsprop', loss = 'categorical_crossentropy', metrics= ['accuracy'])

In [64]:
model.fit(X_train, y_train, batch_size= 32, epochs = 20)

Epoch 1/20
132/132 [==============================] - 1s 5ms/step - loss: 1.5742 - accuracy: 0.2500
Epoch 2/20
132/132 [==============================] - 0s 565us/step - loss: 1.5018 - accuracy: 0.3333
Epoch 3/20
132/132 [==============================] - 0s 541us/step - loss: 1.4296 - accuracy: 0.4015
Epoch 4/20
132/132 [==============================] - 0s 546us/step - loss: 1.3185 - accuracy: 0.4773
Epoch 5/20
132/132 [==============================] - 0s 574us/step - loss: 1.2325 - accuracy: 0.5758
Epoch 6/20
132/132 [==============================] - 0s 654us/step - loss: 1.1405 - accuracy: 0.5909
Epoch 7/20
132/132 [==============================] - 0s 553us/step - loss: 1.0305 - accuracy: 0.6515
Epoch 8/20
132/132 [==============================] - 0s 561us/step - loss: 0.9828 - accuracy: 0.6288
Epoch 9/20
132/132 [==============================] - 0s 593us/step - loss: 0.9079 - accuracy: 0.6439
Epoch 10/20
132/132 [==============================] - 0s 566us/step - loss: 0.8372 

In [69]:
y_pred = model.predict(X_train)

In [70]:
y_pred[:4]

array([[5.2429619e-03, 3.0113971e-03, 2.6582864e-01, 7.2562009e-01,
        2.9690983e-04],
       [5.7857912e-03, 1.3921162e-03, 9.6643358e-01, 2.5847975e-02,
        5.4052460e-04],
       [6.2057734e-03, 1.7281085e-02, 5.8385563e-01, 3.9178514e-01,
        8.7230001e-04],
       [3.0278626e-01, 4.4001895e-03, 2.6626381e-01, 4.2578995e-01,
        7.5979024e-04]], dtype=float32)

In [72]:
y_train[:5]

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [75]:
X = train.iloc[:, 0].values
for i in range(len(y_pred)):
    pred = np.argmax(y_pred[i])
    print(" ".join(X[i]))
    val = np.argmax(y_train[i])
    print(emoji.emojize(emoji_dictionary[str(val)]), end = " --> ")
    print(emoji.emojize(emoji_dictionary[str(pred)]))

never talk to me again
🤐 --> 🤐
I am proud of your achievements
😃 --> 😃
It is the worst day in my life
🤐 --> 😃
Miss you so much
❤️ --> 🤐
food is life
🍴 --> 🍴
I love you mum
❤️ --> ❤️
Stop saying bullshit
🤐 --> 😃
congratulations on your acceptance
😃 --> 😃
The assignment is too long
🤐 --> 😃
I want to go play
⚾ --> ⚾
she did not answer my text
🤐 --> 🤐
Your stupidity has no limit
🤐 --> 🤐
how many points did he score
⚾ --> ⚾
my algorithm performs poorly
🤐 --> 🤐
I got approved
😃 --> 😃
Stop shouting at me
🤐 --> 😃
Sounds like a fun plan ha ha
😃 --> 😃
no one likes him
🤐 --> 😃
the game just finished
⚾ --> ⚾
I will celebrate soon
😃 --> 😃
So sad you are not coming
🤐 --> 🤐
She is my dearest love
❤️ --> ❤️
Good job
😃 --> 😃
It was funny lol
😃 --> 😃
candy is life
😃 --> 😃
The chicago cubs won again
⚾ --> ⚾
I am hungry
🍴 --> 🍴
I am so excited to see you after so long
😃 --> 😃
you did well on you exam
😃 --> 🤐
lets brunch some day
🍴 --> 🍴
he is so cute
❤️ --> 😃
How dare you ask that
🤐 --> 🤐
do you want to j

In [76]:
y_pred.shape

(132, 5)

In [77]:
n = np.argmax(y_pred, axis = 1)

In [78]:
n[:10]

array([3, 2, 2, 3, 4, 0, 2, 2, 2, 1])

In [79]:
n1 = np.argmax(y_train, axis = 1)

In [80]:
from sklearn.metrics import confusion_matrix, classification_report

In [81]:
confusion_matrix(n, n1)

array([[13,  0,  0,  0,  0],
       [ 0, 18,  0,  0,  0],
       [ 5,  0, 37, 11,  0],
       [ 4,  1,  1, 25,  0],
       [ 0,  0,  0,  0, 17]])

In [82]:
print(classification_report(n1, n))

              precision    recall  f1-score   support

           0       1.00      0.59      0.74        22
           1       1.00      0.95      0.97        19
           2       0.70      0.97      0.81        38
           3       0.81      0.69      0.75        36
           4       1.00      1.00      1.00        17

    accuracy                           0.83       132
   macro avg       0.90      0.84      0.86       132
weighted avg       0.86      0.83      0.83       132

